In [1]:
import pandas as pd
import sympy as sy

In [2]:
inputData = pd.read_csv('M+N Example')

In [3]:
run parseInput.py

In [4]:
run calcIsotopologues.py

In [5]:
import basicDeltaOperations as op

def computeMNUValues(MNSolution, key):
    '''
    Given an MN U value, update the MN output dataframe to include clumped and site-specific delta values
    
    Key should be "MN", i.e. "M1".
    '''
    MNSolution['U Values'] = MNSolution[key + ' Percent Abundance'] * MNSolution["U" + key]
    
    #calculate clumped deltas
    clumpedDeltas = [1000*(x/y-1) for x, y in zip(MNSolution['U Values'].values, MNSolution['Stochastic U'].values)]
    clumpedCulled = []
    for i in range(len(clumpedDeltas)):
        if '|' in MNSolution.index[i]:
            clumpedCulled.append(clumpedDeltas[i])
        else:
            clumpedCulled.append('N/A')
      
    #calculate site specific deltas
    deltas = []
    for i, v in MNSolution.iterrows():
        if '|' not in i:
            delta = op.ratioToDelta(v['Composition'],v['U Values'])
            deltas.append(delta)
            
        else:
            deltas.append('N/A')

    MNSolution['Deltas'] = deltas
    MNSolution['Clumped Deltas'] = clumpedCulled
    
    return MNSolution

def GJElim(Matrix, augMatrix = False):
    M = Matrix.copy()
    rows, cols = M.shape

    r = 0
    c = 0
    
    if augMatrix == True:
        colLimit = cols - 1
    else:
        colLimit = cols
        
    rank = 0
    storage = []
    while r < rows and c < colLimit:
        storage.append(M.copy())
        #If there is a nonzero entry in the column, then pivot and eliminate. 
        if True in (M[r:,c]!=0):
            pivotRow = (M[r:,c]!=0).argmax(axis=0) + r
            rank += 1

            M[[r, pivotRow]] = M[[pivotRow, r]]

            M[r] = M[r]/ M[r,c]

            for i in range(1,rows-r):
                M[r+i] -= (M[r+i,c]/M[r,c] * M[r])

            for j in range(0,r):
                M[j] -= M[j,c]/M[r,c] * M[r]
                
            r += 1

        c += 1

    storage.append(M.copy())
        
    return M, rank, storage

Both to simulate a measurement and to reconstruct the molecule based on measurements, we begin by defining basic information about the experiment. This is included in the top block of the .csv file. It includes our sites, the number of atoms present ("Stoich"), and how fragments sample the sites. It also includes site-specific delta values that will be used to simulate the measurements. 

In [6]:
inputFile = parseInput('C2NO2ExampleClump.csv')
df = pd.DataFrame.from_dict(inputFile['basicInfo'])
fragmentList = inputFile['M1Dict']['Fragment List']
df['Stoich'] = [int(x) for x in list(df['Stoich'])]
df.rename(columns={'atom ID':'site',"element": "IDS", "Stoich": "Number",'Ref Deltas':'deltas'},inplace = True)

In [7]:
df

,site,IDS,Number,Equivalence,70,54,42,deltas
0,C-1,C,1,0.0,1.0,1.0,0.0,25.0
1,C-2,C,1,0.0,1.0,1.0,1.0,-25.0
2,N-3,N,1,0.0,1.0,1.0,1.0,0.0
3,O-4,O,1,0.0,1.0,1.0,1.0,13.0
4,O-5,O,1,0.0,1.0,0.0,0.0,-13.0


In [8]:
siteElements = strSiteElements(df)
siteIsotopes, multinomialCoeff = calculateSetsOfSiteIsotopes(df)
bigA, SN = calcAllIsotopologues(siteIsotopes, multinomialCoeff)
concentrationArray = siteSpecificConcentrations(df)
d = calculateIsotopologueConcentrations(bigA, SN, concentrationArray)

byCondensed = {}
siteElements = strSiteElements(df)
for i, v in d.items():
    condensed = condenseStr(i)
    byCondensed[condensed] = {}
    byCondensed[condensed]['Number'] = v['num']
    byCondensed[condensed]['full'] = i
    byCondensed[condensed]['Conc'] = v['Conc']
    byCondensed[condensed]['Mass'] = np.array(list(map(int,condensed))).sum()
    byCondensed[condensed]['Subs'] = ''.join([uEl(element, int(number)) for element, number in zip(siteElements, condensed)])
    
M0 = {}
M1 = {}
M2 = {}
M3 = {}
M4 = {}

for i, v in byCondensed.items():
    if v['Mass'] == 0:
        M0[i] = v
    if v['Mass'] == 1:
        M1[i] = v
    if v['Mass'] == 2:
        M2[i] = v
    if v['Mass'] == 3:
        M3[i] = v
    if v['Mass'] == 4:
        M4[i] = v
    

In [9]:
inputData

,Unnamed: 0,Abs. Abundance,Rel. Abundance,Adj. Rel. Abundance
0,13C U Value,2.247440e-02,0.022474,0.022474
1,15N U Value,3.676000e-03,0.003676,0.003676
2,17O U Value,7.598000e-04,0.000760,0.000760
3,18O U Value,4.010400e-03,0.004010,0.004010
4,13C13C U Value,1.261957e-04,0.000126,0.000126
...,...,...,...,...
73,M4 42 13C/15N/17O,1.787487e-10,0.000038,0.000038
74,M4 42 13C/15N/18O,8.026958e-08,0.016908,0.016908
75,M4 42 17O,8.403385e-09,0.001770,0.001770
76,M4 42 15N,8.026958e-08,0.016908,0.016908


In [10]:
M2Measurements = {}

for i, v in inputData.iterrows():
    frag = v['Unnamed: 0'].split(' ')[1]
    sub = v['Unnamed: 0'].split(' ')[2]
    if sub == '':
        sub = 'Unsub'
            
    if v['Unnamed: 0'][:2] == 'M2':
        if frag not in M2Measurements:
            M2Measurements[frag] = {}
        M2Measurements[frag][sub] = v['Adj. Rel. Abundance']

M2Df = pd.DataFrame.from_dict(M2Measurements)
M2Df.fillna(0, inplace = True)

In [11]:
pd.set_option("precision", 15)
M2Df

,54,42
Unsub,0.461185710910699,0.462204494529890
17O,0.000034039150764,0.001079660028414
18O,0.484836260188171,0.484836260188171
15N,0.000325143802349,0.010312970770481
15N/17O,0.000333708887314,0.000333708887314
13C,0.001987870476470,0.030737674718034
13C/17O,0.002040235858829,0.000994614981179
13C/15N,0.019488442864648,0.009500615896516
13C/13C,0.029768587860755,0.000000000000000


Next, we input measurement information, including observed M2 values. Although this is just a one-line function, quite a bit is happening under the hood. We'll take a closer look at the output of this function...

In [12]:
M2

{'00002': {'Number': 1,
  'full': '00002',
  'Conc': 0.001895804401698337,
  'Mass': 2,
  'Subs': '18O'},
 '00011': {'Number': 1,
  'full': '00011',
  'Conc': 1.3992534964117162e-07,
  'Mass': 2,
  'Subs': '17O17O'},
 '00020': {'Number': 1,
  'full': '00020',
  'Conc': 0.0019930251402469697,
  'Mass': 2,
  'Subs': '18O'},
 '00101': {'Number': 1,
  'full': '00101',
  'Conc': 1.3365744798559682e-06,
  'Mass': 2,
  'Subs': '15N17O'},
 '00110': {'Number': 1,
  'full': '00110',
  'Conc': 1.3717831287680805e-06,
  'Mass': 2,
  'Subs': '15N17O'},
 '01001': {'Number': 1,
  'full': '01001',
  'Conc': 3.983642784660376e-06,
  'Mass': 2,
  'Subs': '13C17O'},
 '01010': {'Number': 1,
  'full': '01010',
  'Conc': 4.08858170299996e-06,
  'Mass': 2,
  'Subs': '13C17O'},
 '01100': {'Number': 1,
  'full': '01100',
  'Conc': 3.905435274630086e-05,
  'Mass': 2,
  'Subs': '13C15N'},
 '10001': {'Number': 1,
  'full': '10001',
  'Conc': 4.187932158232704e-06,
  'Mass': 2,
  'Subs': '13C17O'},
 '10010': {'Num

In [13]:
#Then defining fragments
#parseInput currently returns floats, not 'x'. Need to fix. 
def zeroToX(y):
    if y == 0:
        return 'x'
    else:
        return int(y)
    
f54 = df['54'].values
f42 = df['42'].values


frag_54 = [zeroToX(z) for z in f54]
frag_42 = [zeroToX(z) for z in f42]

fragments = [frag_54, frag_42]
fragKeys = ['54','42']

In [14]:
UnsubConc = M0['00000']['Conc']

#For each fragment we will observe
for j, fragment in enumerate(fragments):
    #compute the isotopologues present after fragmentation and track their concentrations
    fragmentedDict = {}
    for isotopologue, value in M2.items():
        value['Stochastic U'] = value['Conc'] / UnsubConc
        frag = [fragMult(x,y) for x, y in zip(fragment, isotopologue)]
        newIsotopologue = ''.join(frag)
        M2[isotopologue][fragKeys[j] + ' Identity'] = newIsotopologue
        
        sub = computeSubs(newIsotopologue, siteElements)
        
        if sub == '':
            sub = 'Unsub'
            
        M2[isotopologue][fragKeys[j] + ' Subs'] = sub

In [15]:
M0

{'00000': {'Number': 1,
  'full': '00000',
  'Conc': 0.9696862013627834,
  'Mass': 0,
  'Subs': ''}}

In [16]:
def filterEmptyStr(string):
    if string == '':
        return False
    else:
        return True
    
Isotopologues = pd.DataFrame.from_dict(M2).T
Isotopologues.rename(columns={'Conc':'Stochastic',"Subs": "Composition"},inplace = True)
preciseStrings = []
for i, v in Isotopologues.iterrows():
    Subs = [uEl(element, int(number)) for element, number in zip(siteElements, i)]
    Precise = [x + " " + y for x, y in zip(Subs, df['site']) if x != '']
    output = '   |   '.join(Precise)
    preciseStrings.append(output)
Isotopologues['Precise Identity'] = preciseStrings
Isotopologues.sort_values('Composition',inplace = True)

In [17]:
Isotopologues

,Number,full,Stochastic,Mass,Composition,Stochastic U,54 Identity,54 Subs,42 Identity,42 Subs,Precise Identity
11000,1,11000,0.00012237026985793,2,13C13C,0.0001261957421751,1100x,13C/13C,x100x,13C,13C C-1 | 13C C-2
01100,1,01100,3.90543527463009e-05,2,13C15N,4.027524852e-05,0110x,13C/15N,x110x,13C/15N,13C C-2 | 15N N-3
10100,1,10100,4.1057140066624e-05,2,13C15N,4.234064588e-05,1010x,13C/15N,x010x,15N,13C C-1 | 15N N-3
01001,1,01001,3.98364278466038e-06,2,13C17O,4.108177242351e-06,0100x,13C,x100x,13C,13C C-2 | 17O O-5
01010,1,01010,4.08858170299996e-06,2,13C17O,4.216396703649e-06,0101x,13C/17O,x101x,13C/17O,13C C-2 | 17O O-4
10001,1,10001,4.1879321582327e-06,2,13C17O,4.318852998369e-06,1000x,13C,x000x,Unsub,13C C-1 | 17O O-5
10010,1,10010,4.29825255956406e-06,2,13C17O,4.432622175631e-06,1001x,13C/17O,x001x,17O,13C C-1 | 17O O-4
00101,1,00101,1.33657447985597e-06,2,15N17O,1.3783577388e-06,0010x,15N,x010x,15N,15N N-3 | 17O O-5
00110,1,00110,1.37178312876808e-06,2,15N17O,1.4146670612e-06,0011x,15N/17O,x011x,15N/17O,15N N-3 | 17O O-4
00011,1,00011,1.39925349641172e-07,2,17O17O,1.4429961924231e-07,0001x,17O,x001x,17O,17O O-4 | 17O O-5


In [18]:
M2Df

,54,42
Unsub,0.461185710910699,0.462204494529890
17O,0.000034039150764,0.001079660028414
18O,0.484836260188171,0.484836260188171
15N,0.000325143802349,0.010312970770481
15N/17O,0.000333708887314,0.000333708887314
13C,0.001987870476470,0.030737674718034
13C/17O,0.002040235858829,0.000994614981179
13C/15N,0.019488442864648,0.009500615896516
13C/13C,0.029768587860755,0.000000000000000


In [19]:
CMatrix = []
MeasurementVector = []

closure = np.ones(len(Isotopologues['Number']),dtype = int)
CMatrix.append(closure)
MeasurementVector.append(1)

for fragment in fragKeys:
    IsotopologueFragments = Isotopologues[fragment + ' Subs']
    for sub, v in M2Df[fragment].iteritems():
        c = list(IsotopologueFragments.isin([sub]) * 1)
        CMatrix.append(c)
        MeasurementVector.append(v)

In [20]:
sy.Matrix(CMatrix)

Matrix([
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [21]:
sy.Matrix(MeasurementVector)

Matrix([
[                   1],
[   0.461185710910699],
[   3.403915076412e-5],
[   0.484836260188171],
[0.000325143802348633],
[0.000333708887314251],
[ 0.00198787047647011],
[   0.002040235858829],
[   0.019488442864648],
[  0.0297685878607553],
[    0.46220449452989],
[ 0.00107966002841398],
[   0.484836260188171],
[  0.0103129707704807],
[0.000333708887314251],
[  0.0307376747180345],
[0.000994614981179136],
[ 0.00950061589651587],
[                 0.0]])

We also set up the matrix inversion problem we must solve. We define a "Composition Matrix", where each row corresponds to an individual measurement and each column corresponds to an isotopologue. We also define a "Measurement Vector", where each row gives the result of an individual measurement. The composition matrix takes a vector giving the relative concentrations of each isotopologue in M2 space to the observed measurement; matrix inversion therefore gives the relative concentration of each isotopologue in M2 space.

Additionally, we define "Full Matrix Order" and "Single Fragment Order" vectors. These track what measurement each row of the composition matrix corresponds to. For example, the first row gives closure, the second gives the "18O" substitution of the highest mass fragment, the third gives "13C/13C" substituion of the highest mass fragment, and so forth. Each fragment repeats the same possible substitutions in order. 

We put the information into an augmented matrix to prepare to solve

In [22]:
comp = np.array(CMatrix,dtype=float)
meas = np.array(MeasurementVector,dtype = float)
AugMatrix = np.column_stack((comp, meas))

In [23]:
sy.Matrix(AugMatrix)

Matrix([
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,                  1.0],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,    0.461185710910699],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,    3.403915076412e-5],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,    0.484836260188171],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.000325143802348633],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.000333708887314251],
[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.00198787047647011],
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.002040235858829],
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.019488442864648],
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.0297685878607553],
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,     0.46220449452989],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,  0.001

And solve

In [24]:
solve = GJElim(AugMatrix, augMatrix = True)

In [25]:
sy.Matrix(solve[0])

Matrix([
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.0297685878607552],
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00950061589651588],
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00998782696813208],
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.000969086857279255],
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.000994614981179143],
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00101878361919083],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00104562087764981],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,  0.000325143802348633],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,   0.00033370888731421],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,   3.40391507641646e-5],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,     0.461185710910699],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Then we solve, tracking which sums of isotopologues are constrained. The first entry in "sol" gives the solution. The second gives additional information from the "rref" function in M2Module.py.

In [26]:
#Construct augmented matrix
comp = np.array(CMatrix,dtype=float)
meas = np.array(MeasurementVector,dtype = float)
AugMatrix = np.column_stack((comp, meas))

#solve by Gauss Jordan
solve = GJElim(AugMatrix, augMatrix = True)

#Take everything but the final column, which is just the answer
solution = solve[0][:,:-1]

#Check which isotopologues correspond to which measurements in the answer, and explicitly track them
uniqueAnswers = []
stochasticValues = []
composition = []

rank = solve[1]
for i in range(len(solution)):
    stoch = 0
    c = None
    
    if i >= rank:
        break
        
    rowIsotopologues = []
    for j in range(len(solution[i])):
        if solution[i][j] == 1:
            rowIsotopologues.append(Isotopologues['Precise Identity'][j])

            stoch += Isotopologues['Stochastic U'][j]
            
            if c == None:
                c = Isotopologues['Composition'][j]
            elif c != Isotopologues['Composition'][j]:
                c = c + " & " + Isotopologues['Composition'][j]
            
    uniqueAnswers.append(rowIsotopologues)
    stochasticValues.append(stoch)
    composition.append(c)
    
#take the measured values
values = solve[0][:rank,-1]

condensed = [' & '.join(x) for x in uniqueAnswers]

#output as dataFrame
output = {}
output['M2 Percent Abundance'] = values
output['Stochastic U'] = stochasticValues
output['Composition'] = composition

dfOutput = pd.DataFrame.from_dict(output)
dfOutput.index = condensed

In [27]:
dfOutput

,M2 Percent Abundance,Stochastic U,Composition
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O


In [28]:
#Manual calculation of UM+2
M218O = dfOutput[dfOutput['Composition'] == '18O']['M2 Percent Abundance'].sum()
UM2 = 4.010400e-03 / M218O

In [29]:
dfOutput['UM2'] = UM2
dfOutput

,M2 Percent Abundance,Stochastic U,Composition,UM2
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C,0.004239225010114
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N,0.004239225010114
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N,0.004239225010114
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O,0.004239225010114
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O,0.004239225010114
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O,0.004239225010114
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O,0.004239225010114
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O,0.004239225010114
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O,0.004239225010114
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O,0.004239225010114


In [30]:
computeMNUValues(dfOutput, "M2")

,M2 Percent Abundance,Stochastic U,Composition,UM2,U Values,Deltas,Clumped Deltas
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C,0.004239225010114,0.000126195742175,N/A,-2.55351295663786e-12
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N,0.004239225010114,0.000040275248520,N/A,4.44089209850063e-13
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N,0.004239225010114,0.000042340645880,N/A,-3.33066907387547e-13
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O,0.004239225010114,0.000004108177242,N/A,7.94919685631612e-11
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O,0.004239225010114,0.000004216396704,N/A,6.43929354282591e-12
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O,0.004239225010114,0.000004318852998,N/A,-9.64783808399261e-11
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O,0.004239225010114,0.000004432622176,N/A,-4.79616346638068e-11
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O,0.004239225010114,0.000001378357739,N/A,0
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O,0.004239225010114,0.000001414667061,N/A,-1.2134737659153e-10
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O,0.004239225010114,0.000000144299619,N/A,1.31272770431679e-09


In [31]:
M1Measurements = {}
M2Measurements = {}
M3Measurements = {}
M4Measurements = {}

for i, v in inputData.iterrows():
    frag = v['Unnamed: 0'].split(' ')[1]
    sub = v['Unnamed: 0'].split(' ')[2]
    if sub == '':
        sub = 'Unsub'
            
    if v['Unnamed: 0'][:2] == 'M1':
        if frag not in M1Measurements:
            M1Measurements[frag] = {}
        M1Measurements[frag][sub] = v['Adj. Rel. Abundance']
        
    if v['Unnamed: 0'][:2] == 'M2':
        if frag not in M2Measurements:
            M2Measurements[frag] = {}
        M2Measurements[frag][sub] = v['Adj. Rel. Abundance']
    
    if v['Unnamed: 0'][:2] == 'M3':
        if frag not in M3Measurements:
            M3Measurements[frag] = {}
        M3Measurements[frag][sub] = v['Adj. Rel. Abundance']
        
    if v['Unnamed: 0'][:2] == 'M4':
        if frag not in M4Measurements:
            M4Measurements[frag] = {}
        M4Measurements[frag][sub] = v['Adj. Rel. Abundance']
        
M1Df = pd.DataFrame.from_dict(M1Measurements)
M1Df.fillna(0, inplace = True)
M2Df = pd.DataFrame.from_dict(M2Measurements)
M2Df.fillna(0, inplace = True)
M3Df = pd.DataFrame.from_dict(M3Measurements)
M3Df.fillna(0, inplace = True)
M4Df = pd.DataFrame.from_dict(M4Measurements)
M4Df.fillna(0, inplace = True)

MNDfList = [M1Df, M2Df, M3Df, M4Df]

In [32]:
#Then defining fragments
#parseInput currently returns floats, not 'x'. Need to fix. 
def zeroToX(y):
    if y == 0:
        return 'x'
    else:
        return int(y)
    
f54 = df['54'].values
f42 = df['42'].values


frag_54 = [zeroToX(z) for z in f54]
frag_42 = [zeroToX(z) for z in f42]

fragments = [frag_54, frag_42]
fragKeys = ['54','42']

In [33]:
UnsubConc = M0['00000']['Conc']

#For each fragment we will observe
massSelected = [M1, M2, M3, M4]
massKeys = ['M1','M2','M3','M4']

for selection in massSelected:
    for j, fragment in enumerate(fragments):
        #compute the isotopologues present after fragmentation and track their concentrations
        fragmentedDict = {}
        for isotopologue, value in selection.items():
            value['Stochastic U'] = value['Conc'] / UnsubConc
            frag = [fragMult(x,y) for x, y in zip(fragment, isotopologue)]
            newIsotopologue = ''.join(frag)
            selection[isotopologue][fragKeys[j] + ' Identity'] = newIsotopologue

            sub = computeSubs(newIsotopologue, siteElements)

            if sub == '':
                sub = 'Unsub'

            selection[isotopologue][fragKeys[j] + ' Subs'] = sub

In [34]:
def filterEmptyStr(string):
    if string == '':
        return False
    else:
        return True
    
IsotopologuesBySelection = {}

for index, selection in enumerate(massSelected):
    Isotopologues = pd.DataFrame.from_dict(selection).T
    Isotopologues.rename(columns={'Conc':'Stochastic',"Subs": "Composition"},inplace = True)
    preciseStrings = []
    for i, v in Isotopologues.iterrows():
        Subs = [uEl(element, int(number)) for element, number in zip(siteElements, i)]
        Precise = [x + " " + y for x, y in zip(Subs, df['site']) if x != '']
        output = '   |   '.join(Precise)
        preciseStrings.append(output)
    Isotopologues['Precise Identity'] = preciseStrings
    Isotopologues.sort_values('Composition',inplace = True)
    
    IsotopologuesBySelection[massKeys[index]] = Isotopologues

In [35]:
IsotopologuesBySelection['M4']

,Number,full,Stochastic,Mass,Composition,Stochastic U,54 Identity,54 Subs,42 Identity,42 Subs,Precise Identity
11110,1,11110,1.73113190038169e-10,4,13C13C15N17O,1.78524959718802e-10,1111x,13C/13C/15N/17O,x111x,13C/15N/17O,13C C-1 | 13C C-2 | 15N N-3 | 17O O-4
11101,1,11101,1.68670008457722e-10,4,13C13C15N17O,1.73942877830659e-10,1110x,13C/13C/15N,x110x,13C/15N,13C C-1 | 13C C-2 | 15N N-3 | 17O O-5
11011,1,11011,1.7657983347078e-11,4,13C13C17O17O,1.82099975458677e-11,1101x,13C/13C/17O,x101x,13C/17O,13C C-1 | 13C C-2 | 17O O-4 | 17O O-5
11020,1,11020,2.51511286747099e-07,4,13C13C18O,2.59373894764748e-07,1102x,13C/13C/18O,x102x,13C/18O,13C C-1 | 13C C-2 | 18O O-4
11002,1,11002,2.39242443491143e-07,4,13C13C18O,2.46721509654273e-07,1100x,13C/13C,x100x,13C,13C C-1 | 13C C-2 | 18O O-5
01111,1,01111,5.63552823104608e-12,4,13C15N17O17O,5.81170302632541e-12,0111x,13C/15N/17O,x111x,13C/15N/17O,13C C-2 | 15N N-3 | 17O O-4 | 17O O-5
10111,1,10111,5.92452967879203e-12,4,13C15N17O17O,6.10973907895748e-12,1011x,13C/15N/17O,x011x,15N/17O,13C C-1 | 15N N-3 | 17O O-4 | 17O O-5
10120,1,10120,8.43859716931343e-08,4,13C15N18O,8.70239996965404e-08,1012x,13C/15N/18O,x012x,15N/18O,13C C-1 | 15N N-3 | 18O O-4
01102,1,01102,7.63539934237104e-08,4,13C15N18O,7.87409301239964e-08,0110x,13C/15N,x110x,13C/15N,13C C-2 | 15N N-3 | 18O O-5
01120,1,01120,8.02695828300545e-08,4,13C15N18O,8.27789265406116e-08,0112x,13C/15N/18O,x112x,13C/15N/18O,13C C-2 | 15N N-3 | 18O O-4


In [36]:
solved = []
for index, selection in enumerate(massSelected):
    Isotopologues = IsotopologuesBySelection[massKeys[index]]
    MNDf = MNDfList[index]
    
    CMatrix = []
    MeasurementVector = []

    closure = np.ones(len(Isotopologues['Number']),dtype = int)
    CMatrix.append(closure)
    MeasurementVector.append(1)

    for fragment in fragKeys:
        IsotopologueFragments = Isotopologues[fragment + ' Subs']
        for sub, v in MNDf[fragment].iteritems():
            c = list(IsotopologueFragments.isin([sub]) * 1)
            CMatrix.append(c)
            MeasurementVector.append(v)
            
    #Construct augmented matrix
    comp = np.array(CMatrix,dtype=float)
    meas = np.array(MeasurementVector,dtype = float)
    AugMatrix = np.column_stack((comp, meas))

    #solve by Gauss Jordan
    solve = GJElim(AugMatrix, augMatrix = True)

    if index == 0:
        b = solve
    #Take everything but the final column, which is just the answer
    solution = solve[0][:,:-1]

    #Check which isotopologues correspond to which measurements in the answer, and explicitly track them
    uniqueAnswers = []
    stochasticValues = []
    composition = []

    rank = solve[1]
    for i in range(len(solution)):
        stoch = 0
        c = None

        if i >= rank:
            break

        rowIsotopologues = []
        for j in range(len(solution[i])):
            if solution[i][j] == 1:
                rowIsotopologues.append(Isotopologues['Precise Identity'][j])

                stoch += Isotopologues['Stochastic U'][j]

                if c == None:
                    c = Isotopologues['Composition'][j]
                elif c != Isotopologues['Composition'][j]:
                    c = c + " & " + Isotopologues['Composition'][j]

        uniqueAnswers.append(rowIsotopologues)
        stochasticValues.append(stoch)
        composition.append(c)

    #take the measured values
    values = solve[0][:rank,-1]

    condensed = [' & '.join(x) for x in uniqueAnswers]

    #output as dataFrame
    output = {}
    output[massKeys[index] + ' Percent Abundance'] = values
    output['Stochastic U'] = stochasticValues
    output['Composition'] = composition

    dfOutput = pd.DataFrame.from_dict(output)
    dfOutput.index = condensed
    
    solved.append(dfOutput)

In [37]:
sy.Matrix(b[2][5])

Matrix([
[1.0, 0.0, 0.0, 0.0, 0.0,    0.407141901583786],
[0.0, 1.0, 0.0, 0.0, 0.0,    0.428020973459878],
[0.0, 0.0, 1.0, 0.0, 0.0,    0.136602477870845],
[0.0, 0.0, 0.0, 1.0, 0.0,   0.0139337983366902],
[0.0, 0.0, 0.0, 0.0, 1.0,   0.0143008487488016],
[0.0, 0.0, 0.0, 0.0, 0.0, 2.25514051876985e-17],
[0.0, 0.0, 0.0, 0.0, 0.0,                  0.0],
[0.0, 0.0, 0.0, 0.0, 0.0,                  0.0],
[0.0, 0.0, 0.0, 0.0, 0.0, 7.11236625150491e-17]])

In [38]:
M1Solution = solved[0]
M1Solution

,M1 Percent Abundance,Stochastic U,Composition
13C C-2,0.407141901583786,0.0109562700,13C
13C C-1,0.428020973459878,0.0115181300,13C
15N N-3,0.136602477870845,0.0036760000,15N
17O O-5,0.013933798336690,0.0003749613,17O
17O O-4,0.014300848748802,0.0003848387,17O


In [39]:
#Manual calculation of UM+1
M113C = M1Solution[M1Solution['Composition'] == '13C']['M1 Percent Abundance'].sum()
UM1 = 0.022474400000000 / M113C
M1Solution['UM1'] = UM1
computeMNUValues(M1Solution, "M1")

,M1 Percent Abundance,Stochastic U,Composition,UM1,U Values,Deltas,Clumped Deltas
13C C-2,0.407141901583786,0.0109562700,13C,0.0269102,0.0109562700,-25.000000000000355,N/A
13C C-1,0.428020973459878,0.0115181300,13C,0.0269102,0.0115181300,25.000000000000135,N/A
15N N-3,0.136602477870845,0.0036760000,15N,0.0269102,0.0036760000,0.000000000000444,N/A
17O O-5,0.013933798336690,0.0003749613,17O,0.0269102,0.0003749613,-13.000000000001567,N/A
17O O-4,0.014300848748802,0.0003848387,17O,0.0269102,0.0003848387,12.999999999999901,N/A


In [40]:
M2Solution = solved[1]
M2Solution

,M2 Percent Abundance,Stochastic U,Composition
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O


In [41]:
#Manual calculation of UM+1
M218O = M2Solution[M2Solution['Composition'] == '18O']['M2 Percent Abundance'].sum()
UM2 = 4.010400e-03 / M218O
M2Solution['UM2'] = UM2
computeMNUValues(M2Solution, "M2")

,M2 Percent Abundance,Stochastic U,Composition,UM2,U Values,Deltas,Clumped Deltas
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C,0.004239225010114,0.000126195742175,N/A,-2.55351295663786e-12
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N,0.004239225010114,0.000040275248520,N/A,4.44089209850063e-13
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N,0.004239225010114,0.000042340645880,N/A,-3.33066907387547e-13
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O,0.004239225010114,0.000004108177242,N/A,7.94919685631612e-11
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O,0.004239225010114,0.000004216396704,N/A,6.43929354282591e-12
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O,0.004239225010114,0.000004318852998,N/A,-9.64783808399261e-11
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O,0.004239225010114,0.000004432622176,N/A,-4.79616346638068e-11
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O,0.004239225010114,0.000001378357739,N/A,0
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O,0.004239225010114,0.000001414667061,N/A,-1.2134737659153e-10
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O,0.004239225010114,0.000000144299619,N/A,1.31272770431679e-09


In [42]:
M3Solution = solved[2]
M3Solution

,M3 Percent Abundance,Stochastic U,Composition
13C C-1 | 13C C-2 | 15N N-3,0.004334542954196,0.000000463895548,13C13C15N
13C C-1 | 13C C-2 | 17O O-5,0.000442134347391,0.000000047318520,13C13C17O
13C C-1 | 13C C-2 | 17O O-4,0.000453781250160,0.000000048565005,13C13C17O
13C C-1 | 15N N-3 | 17O O-4,0.000152250709008,0.000000016294319,13C15N17O
13C C-1 | 15N N-3 | 17O O-5,0.000148342990909,0.000000015876104,13C15N17O
13C C-2 | 15N N-3 | 17O O-4,0.000144823845154,0.000000015499474,13C15N17O
13C C-2 | 15N N-3 | 17O O-5,0.000141106747450,0.000000015101660,13C15N17O
13C C-1 | 17O O-4 | 17O O-5,0.000015529957501,0.000000001662062,13C17O17O
13C C-2 | 17O O-4 | 17O O-5,0.000014772398599,0.000000001580986,13C17O17O
13C C-2 | 18O O-4,0.210410492917279,0.000022518750419,13C18O


In [43]:
M3C18O = M3Solution[M3Solution['Composition'] == '13C18O']['M3 Percent Abundance'].sum()
UM3 = 0.000090131333760 / M3C18O
M3Solution['UM3'] = UM3
computeMNUValues(M3Solution, "M3")

,M3 Percent Abundance,Stochastic U,Composition,UM3,U Values,Deltas,Clumped Deltas
13C C-1 | 13C C-2 | 15N N-3,0.004334542954196,0.000000463895548,13C13C15N,0.000107022944088,0.000000463895548,N/A,-0.000000000001221
13C C-1 | 13C C-2 | 17O O-5,0.000442134347391,0.000000047318520,13C13C17O,0.000107022944088,0.000000047318520,N/A,0.000000000000222
13C C-1 | 13C C-2 | 17O O-4,0.000453781250160,0.000000048565005,13C13C17O,0.000107022944088,0.000000048565005,N/A,0.000000000000222
13C C-1 | 15N N-3 | 17O O-4,0.000152250709008,0.000000016294319,13C15N17O,0.000107022944088,0.000000016294319,N/A,0.000000000000222
13C C-1 | 15N N-3 | 17O O-5,0.000148342990909,0.000000015876104,13C15N17O,0.000107022944088,0.000000015876104,N/A,-0.000000000100919
13C C-2 | 15N N-3 | 17O O-4,0.000144823845154,0.000000015499474,13C15N17O,0.000107022944088,0.000000015499474,N/A,0.000000000000666
13C C-2 | 15N N-3 | 17O O-5,0.000141106747450,0.000000015101660,13C15N17O,0.000107022944088,0.000000015101660,N/A,0.000000000081712
13C C-1 | 17O O-4 | 17O O-5,0.000015529957501,0.000000001662062,13C17O17O,0.000107022944088,0.000000001662062,N/A,-0.000000009457324
13C C-2 | 17O O-4 | 17O O-5,0.000014772398599,0.000000001580986,13C17O17O,0.000107022944088,0.000000001580986,N/A,0.000000009922951
13C C-2 | 18O O-4,0.210410492917279,0.000022518750419,13C18O,0.000107022944088,0.000022518750419,N/A,0.000000000000222


In [44]:
M4Solution = solved[3]
M4Solution

,M4 Percent Abundance,Stochastic U,Composition
13C C-1 | 13C C-2 | 15N N-3 | 17O O-4,0.000036463853388,0.000000000178525,13C13C15N17O
13C C-1 | 13C C-2 | 15N N-3 | 17O O-5,0.000035527959817,0.000000000173943,13C13C15N17O
13C C-1 | 13C C-2 | 17O O-4 | 17O O-5,0.000003719405296,0.000000000018210,13C13C17O17O
13C C-1 | 13C C-2 | 18O O-4,0.052977307410479,0.000000259373895,13C13C18O
13C C-1 | 13C C-2 | 18O O-5,0.050393048512407,0.000000246721510,13C13C18O
13C C-2 | 15N N-3 | 17O O-4 | 17O O-5,0.000001187044587,0.000000000005812,13C15N17O17O
13C C-1 | 15N N-3 | 17O O-4 | 17O O-5,0.000001247918669,0.000000000006110,13C15N17O17O
13C C-1 | 15N N-3 | 18O O-4,0.017774715486285,0.000000087024000,13C15N18O
13C C-2 | 15N N-3 | 18O O-5,0.016082892477477,0.000000078740930,13C15N18O
13C C-2 | 15N N-3 | 18O O-4,0.016907656194271,0.000000082778927,13C15N18O


In [45]:
M418O18O = M4Solution[M4Solution['Composition'] == '18O18O']['M4 Percent Abundance'].sum()
UM4 = 0.000004018314023 / M418O18O
M4Solution['UM4'] = UM4
computeMNUValues(M4Solution, "M4")

,M4 Percent Abundance,Stochastic U,Composition,UM4,U Values,Deltas,Clumped Deltas
13C C-1 | 13C C-2 | 15N N-3 | 17O O-4,0.000036463853388,0.000000000178525,13C13C15N17O,0.000004895943328,0.000000000178525,N/A,-0.000000023370195
13C C-1 | 13C C-2 | 15N N-3 | 17O O-5,0.000035527959817,0.000000000173943,13C13C15N17O,0.000004895943328,0.000000000173943,N/A,-0.000000024886093
13C C-1 | 13C C-2 | 17O O-4 | 17O O-5,0.000003719405296,0.000000000018210,13C13C17O17O,0.000004895943328,0.000000000018210,N/A,-0.000000024886093
13C C-1 | 13C C-2 | 18O O-4,0.052977307410479,0.000000259373895,13C13C18O,0.000004895943328,0.000000259373895,N/A,-0.000000024886093
13C C-1 | 13C C-2 | 18O O-5,0.050393048512407,0.000000246721510,13C13C18O,0.000004895943328,0.000000246721510,N/A,-0.000000024886315
13C C-2 | 15N N-3 | 17O O-4 | 17O O-5,0.000001187044587,0.000000000005812,13C15N17O17O,0.000004895943328,0.000000000005812,N/A,-0.000000106487041
13C C-1 | 15N N-3 | 17O O-4 | 17O O-5,0.000001247918669,0.000000000006110,13C15N17O17O,0.000004895943328,0.000000000006110,N/A,0.000000030383474
13C C-1 | 15N N-3 | 18O O-4,0.017774715486285,0.000000087024000,13C15N18O,0.000004895943328,0.000000087024000,N/A,-0.000000024886093
13C C-2 | 15N N-3 | 18O O-5,0.016082892477477,0.000000078740930,13C15N18O,0.000004895943328,0.000000078740930,N/A,-0.000000024883762
13C C-2 | 15N N-3 | 18O O-4,0.016907656194271,0.000000082778927,13C15N18O,0.000004895943328,0.000000082778927,N/A,-0.000000024885982
